# Time-Series Analysis of Patient Monitoring Data
### Predicting Heart Rate Anomalies using ARIMA and LSTM

In this notebook, we:
- Simulate patient heart rate data over time.
- Identify and visualize an anomaly (simulated event).
- Apply **ARIMA** for time-series forecasting.
- Apply **LSTM** (deep learning) for sequential prediction.

This tutorial demonstrates key concepts in time-series analysis for healthcare applications.

## Step 1: Import Required Libraries

We first import the necessary Python libraries.

In [ ]:
!pip install vitaldb

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# For ARIMA
from statsmodels.tsa.arima.model import ARIMA

# For LSTM
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Set plot style
sns.set(style='whitegrid')

## Step 2: Simulate Patient Heart Rate Data

We generate 2 days of minute-by-minute heart rate data with an anomaly.

In [ ]:
# Create timestamps (every minute for 2 days)
date_range = pd.date_range(start='2023-01-01', periods=2880, freq='T')

# Generate normal heart rate values (80 bpm ± some noise)
np.random.seed(42)
heart_rate = np.random.normal(loc=80, scale=3, size=2880)

# Introduce an anomaly (sudden increase in heart rate)
heart_rate[1400:1500] += 15

# Create DataFrame
df = pd.DataFrame({'timestamp': date_range, 'heart_rate': heart_rate})
df.set_index('timestamp', inplace=True)

# Display first few rows
df.head()

## Step 3: Visualize the Data

Plot the heart rate data with the anomaly highlighted.

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df.index, df['heart_rate'], label='Heart Rate')
plt.axvspan(df.index[1400], df.index[1500], color='red', alpha=0.3, label='Anomaly Period')
plt.title('Simulated Patient Heart Rate over Time')
plt.xlabel('Time')
plt.ylabel('Heart Rate (bpm)')
plt.legend()
plt.show()

## Step 4: Forecasting with ARIMA

We fit an ARIMA model and predict the next 100 minutes of heart rate.

In [ ]:
# Fit ARIMA model
model = ARIMA(df['heart_rate'], order=(5,1,0))
model_fit = model.fit()
print(model_fit.summary())

# Forecast the next 100 minutes
forecast = model_fit.forecast(steps=100)

# Plot
plt.figure(figsize=(12,6))
plt.plot(df.index, df['heart_rate'], label='Observed Heart Rate')
forecast_index = pd.date_range(start=df.index[-1], periods=101, freq='T')[1:]
plt.plot(forecast_index, forecast, label='ARIMA Forecast', color='orange')
plt.title('ARIMA Forecast of Patient Heart Rate')
plt.xlabel('Time')
plt.ylabel('Heart Rate (bpm)')
plt.legend()
plt.show()

## Step 5: Forecasting with LSTM

### 5.1 Data Preparation

We scale the data and create sequences for training the LSTM model.

In [ ]:
# Scale data
scaler = MinMaxScaler(feature_range=(0,1))
data_scaled = scaler.fit_transform(df[['heart_rate']])

# Create sequences (past 60 minutes → next minute prediction)
sequence_length = 60
X, y = [], []
for i in range(len(data_scaled) - sequence_length):
    X.append(data_scaled[i:i+sequence_length])
    y.append(data_scaled[i+sequence_length])
X, y = np.array(X), np.array(y)

# Train-test split
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

### 5.2 Train the LSTM Model

In [ ]:
# Define LSTM model
model_lstm = Sequential([
    LSTM(50, activation='relu', input_shape=(sequence_length, 1)),
    Dense(1)
])
model_lstm.compile(optimizer='adam', loss='mse')

# Train the model
history = model_lstm.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

### 5.3 Evaluate LSTM Predictions

In [ ]:
# Generate predictions
predictions = model_lstm.predict(X_test)

# Inverse transform predictions
predictions_inv = scaler.inverse_transform(predictions)
y_test_inv = scaler.inverse_transform(y_test)

# Plot actual vs predicted values
plt.figure(figsize=(12,6))
plt.plot(y_test_inv, label='Actual Heart Rate')
plt.plot(predictions_inv, label='Predicted Heart Rate', alpha=0.7)
plt.title('LSTM: Actual vs. Predicted Heart Rate')
plt.xlabel('Time Step')
plt.ylabel('Heart Rate (bpm)')
plt.legend()
plt.show()

## Step 6: Conclusion

- **We simulated heart rate data** and introduced an anomaly.
- **We used ARIMA to forecast time-series data.**
- **We built an LSTM model for sequential forecasting.**

**Next Steps:**
- Try different ARIMA and LSTM hyperparameters.
- Experiment with real ICU datasets.
- Implement real-time anomaly detection techniques.